In [ ]:
!pip -q install "transformers>=4.44.0" "accelerate>=0.34.0" "bitsandbytes>=0.44.1" "torch>=2.3.0" "einops" "sentencepiece"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.9 MB/s eta 0:00:00


In [ ]:
MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"  # change here if you like


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

dtype = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8 else torch.float16

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True, trust_remote_code=True)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=dtype,
    load_in_4bit=True,                  # 4-bit quantization (bitsandbytes)
    bnb_4bit_compute_dtype=dtype,
    bnb_4bit_quant_type="nf4",
    trust_remote_code=True
)
model.eval()

def chat(
    messages,
    max_new_tokens=100,
    temperature=0.2,
    top_p=0.95,
    presence_penalty=0.0,
    frequency_penalty=0.0,
):
    """
    messages: list[dict] with roles: "system" | "user" | "assistant"
    returns: generated assistant string
    """
    # Qwen uses a chat template in its tokenizer
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=(temperature > 0),
            repetition_penalty=1.05,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )

    # Slice off the prompt portion
    gen_ids = output_ids[0, input_ids.shape[-1]:]
    text = tokenizer.decode(gen_ids, skip_special_tokens=True)
    return text.strip()


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [ ]:
messages = [
    {"role": "system", "content": "You are a careful reasoning assistant. Think step by step, but keep explanations concise unless detail is necessary."},
    {"role": "user", "content": "Solve: A rectangle has perimeter 50 and one side is 12. What is the area?"}
]
print(chat(messages, max_new_tokens=256, temperature=0.2))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Let's solve this step-by-step:

1. First, recall the formula for the perimeter of a rectangle:
   P = 2(l + w)
   where P is the perimeter, l is the length, and w is the width.

2. We're given that the perimeter (P) is 50 and one side (let's assume it's the length l) is 12.
   So, we can write: 50 = 2(12 + w)

3. Simplify the equation:
   50 = 24 + 2w

4. Subtract 24 from both sides:
   26 = 2w

5. Divide by 2:
   w = 13

6. Now we have the dimensions of the rectangle: length = 12, width = 13

7. To find the area, use the formula:
   Area = l × w
   Area = 12 × 13

8. Calculate the area:
   Area = 156

Therefore, the area of the rectangle is 156 square units.


## Steer reasoning


In [ ]:

!pip -q install "transformers>=4.44.0" "accelerate>=0.34.0" "bitsandbytes>=0.44.1" "torch>=2.3.0" "einops" "sentencepiece"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.4 MB/s eta 0:00:00


In [ ]:
import torch, math, re
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.bfloat16 if (device == "cuda" and torch.cuda.get_device_capability(0)[0] >= 8) else torch.float16

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=dtype,
    load_in_4bit=True,
    bnb_4bit_compute_dtype=dtype,
    bnb_4bit_quant_type="nf4",
    trust_remote_code=True
)
model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm

In [ ]:
def build_inputs(messages):
    return tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

@torch.inference_mode()
def generate_from_messages(messages, max_new_tokens=256, temperature=0.2, top_p=0.95):
    input_ids = build_inputs(messages)
    out_ids = model.generate(
        input_ids=input_ids,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        do_sample=(temperature>0),
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.05
    )
    gen_ids = out_ids[0, input_ids.shape[-1]:]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()


In [ ]:
tasks = [
    "Compute 24 * 36.",
    "If 3x + 5 = 20, solve for x.",
    "A train leaves at 3pm going 60 mph, another at 4pm going 80 mph. When do they meet?"
]

def build_cot_messages(question):
    return [
        {"role":"system","content":"You are a careful reasoning assistant. Always think step by step. Wrap internal reasoning in <thought>...</thought>."},
        {"role":"user","content": f"{question} Show your reasoning inside <thought>...</thought> before the final answer."}
    ]

def build_nocot_messages(question):
    return [
        {"role":"system","content":"You are concise. Give only the final answer, no reasoning."},
        {"role":"user","content": f"{question} Only output the final answer."}
    ]


In [ ]:
@torch.inference_mode()
def get_cot_outputs(tasks):
    outs = []
    for q in tasks:
        msgs = build_cot_messages(q)
        text = generate_from_messages(msgs, max_new_tokens=256, temperature=0.2)
        outs.append((q, msgs, text))
    return outs

cot_ref = get_cot_outputs(tasks)
for q, msgs, text in cot_ref:
    print("Q:", q)
    print("---- CoT output ----")
    print(text[:400], "\n")


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Q: Compute 24 * 36.
---- CoT output ----
<thought>To compute 24 * 36, we can use the distributive property of multiplication over addition. We can break down 24 and 36 into simpler parts:

24 can be thought of as 20 + 4
36 can be thought of as 30 + 6

Using these, we can expand the multiplication as follows:

24 * 36 = (20 + 4) * (30 + 6)
= 20*30 + 20*6 + 4*30 + 4*6

Now we calculate each part:
20*30 = 600
20*6 = 120
4*30 = 120
4*6 = 24
 

Q: If 3x + 5 = 20, solve for x.
---- CoT output ----
<thought>First, I need to isolate x on one side of the equation. To do this, I'll start by subtracting 5 from both sides of the equation to get rid of the constant on the left side.

Starting with:
3x + 5 = 20

Subtract 5 from both sides:
3x + 5 - 5 = 20 - 5
3x = 15

Now that I have isolated the term with x, I can divide both sides by 3 to solve for x:

3x / 3 = 15 / 3
x = 5</thought>

The solutio 

Q: A train leaves at 3pm going 60 mph, another at 4pm going 80 mph. When do they meet?
---- CoT outp

In [ ]:
def get_decoder_layers(m):
    if hasattr(m, "model") and hasattr(m.model, "layers"):
        return m.model.layers
    if hasattr(m, "transformer") and hasattr(m.transformer, "h"):
        return m.transformer.h
    raise ValueError("Could not find decoder layers list.")

layers = get_decoder_layers(model)
num_layers = len(layers)
hidden_size = model.config.hidden_size

STEER_LAYER_IDX = num_layers // 2
print("Using layer", STEER_LAYER_IDX, "of", num_layers)


Using layer 14 of 28


In [ ]:
class Capture:
    def __init__(self, layer):
        self.layer = layer
        self.hiddens = []
    def _hook(self, module, inp, out):
        self.hiddens.append(out.detach())
        return out
    def __enter__(self):
        self.h = self.layer.register_forward_hook(self._hook)
        return self
    def __exit__(self, *args):
        self.h.remove()

def build_prompt_text(messages):
    return tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

def encode_with_hook(full_text, layer):
    toks = tokenizer(full_text, return_tensors="pt").to(model.device)
    with torch.inference_mode(), Capture(layer) as cap:
        _ = model(**toks)
    return toks, cap.hiddens[-1][0]  # (seq, hidden)


In [ ]:
import re

def find_span_indices(text, start_tag="<thought>", end_tag="</thought>"):
    m1, m2 = re.search(re.escape(start_tag), text), re.search(re.escape(end_tag), text)
    if not m1 or not m2: return None
    return m1.end(), m2.start()

def mean_over_span(hidden, start, end):
    start, end = max(start,0), min(end, hidden.shape[0])
    return hidden[start:end].mean(0) if end>start else None

cot_means, nocot_means = [], []

for q, msgs_cot, cot_text in cot_ref:
    span = find_span_indices(cot_text)
    if not span:
        print("[skip]", q)
        continue

    full_text = build_prompt_text(msgs_cot) + cot_text
    toks, acts = encode_with_hook(full_text, layers[STEER_LAYER_IDX])
    total_toks = toks["input_ids"].shape[1]
    # crude but works: take last ~60 tokens of the answer as reasoning span proxy
    v_cot = acts[-60:].mean(0)
    cot_means.append(v_cot)

    msgs_nocot = build_nocot_messages(q)
    text_nc = build_prompt_text(msgs_nocot) + " The answer is"
    toks_nc, acts_nc = encode_with_hook(text_nc, layers[STEER_LAYER_IDX])
    v_nc = acts_nc[-30:].mean(0)
    nocot_means.append(v_nc)

print(f"Collected {len(cot_means)} CoT and {len(nocot_means)} NoCoT samples.")


[skip] A train leaves at 3pm going 60 mph, another at 4pm going 80 mph. When do they meet?
Collected 2 CoT and 2 NoCoT samples.


In [ ]:
def robust_mean(vectors):
    return torch.stack(vectors).mean(0)

mu_cot, mu_nc = robust_mean(cot_means), robust_mean(nocot_means)
steer_vec = mu_cot - mu_nc
steer_vec = steer_vec / (steer_vec.norm() + 1e-9)
steer_vec = steer_vec.to(model.device)
print("Steering vector norm:", float(steer_vec.norm()))


Steering vector norm: 1.0


In [ ]:
class SteeringHook:
    def __init__(self, layer, steer_vec, alpha=0.0):
        self.layer, self.vec, self.alpha = layer, steer_vec, alpha
    def _hook(self, module, inp, out):
        if self.alpha != 0.0:
            out[:, -1, :] += self.alpha * self.vec
        return out
    def __enter__(self):
        self.h = self.layer.register_forward_hook(self._hook)
        return self
    def __exit__(self, *a):
        self.h.remove()

def steered_generate(messages, alpha=0.0, max_new_tokens=200, temperature=0.2):
    input_ids = build_inputs(messages)
    with SteeringHook(layers[STEER_LAYER_IDX], steer_vec, alpha):
        out_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=0.95,
            do_sample=(temperature>0),
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )
    gen_ids = out_ids[0, input_ids.shape[-1]:]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()


In [ ]:
rectangle_messages = [
    {"role":"system","content":"You are a math tutor. Be correct and clear."},
    {"role":"user","content":"A rectangle has perimeter 50 and one side is 12. What is the area?"}
]

for alpha in [-2.0, -1.0, -0.5, 0.0, 0.5, 1.0, 2.0]:
    print(f"\n========== α = {alpha} ==========")
    print(steered_generate(rectangle_messages, alpha=alpha))


In [ ]:
print(steered_generate(rectangle_messages, alpha=60))

To find the area, we need to use the information about the sides of the rectangle. The perimeter is given by:

\[ P = 2 \times (length + width) \]

Since we know the length and width, we can solve for each side:

\[ length = \frac{P}{2} = \frac{2 \times (length + width)} = \frac{2 \times (length + width)} = \frac{2 \times (length + width)} = \frac{2 \times (length + width)} = \frac{2 \times (length + width)} = \frac{2 \times (length + width)} = \frac{2 \times (length + width)} = \frac{2 \times (length + width)} = \frac{2 \ \times (length + width)} = \frac{2 \ \times (length + width)} = \frac{2 \ \times (length + width)} = \
